In [ ]:
%matplotlib inline
import json
from pathlib import Path

import pandas
import numpy as np
import matplotlib.pyplot as plt
import mpld3
mpld3.enable_notebook()

In [ ]:
def load_runinfo(p: Path):
    hard_info = p / "run.json"
    with hard_info.open() as f:
        j = json.load(f)
        gpumodel = j['host']['gpus']['gpus'][0]['model']
        return gpumodel

def load_config(p: Path):
    config_info = p / "config.json"
    with config_info.open() as f:
        j = json.load(f)
        batch_size = j['batch_size']
        ngpu = j['ngpu']
        framework = j['framework']
        return framework, batch_size, ngpu

def load_results(p: Path):
    results_info = p / "results.json"
    with results_info.open() as f:
        j = json.load(f)
        total = j['total']
        time_series = j['time_series']
        return time_series, total

def get_simple_results(_framework):
    p = Path("../results")

    target_gpu = "GeForce GTX 1080 Ti"
    results = []
    ngpu_set = set()
    batch_size_set = set()
    ignore_list = ["_sources", ".gitkeep"]
    for i in p.iterdir():
        if i.name in ignore_list:
            continue
        gpumodel = load_runinfo(i)
        if target_gpu != gpumodel:
            continue
        try:
            framework, batch_size, ngpu = load_config(i)
            if framework != _framework:
                continue
        except:
            continue
        try:
            ts, total = load_results(i)
        except:
            continue
        batch_size_set.add(batch_size)
        ngpu_set.add(ngpu)
        keyname = "{:06d}-{}".format(batch_size, ngpu)
        results.append([framework, keyname, total, ts])


    batch_size_list = list(batch_size_set) 
    batch_size_list.sort()
    ngpu_list = list(ngpu_set) 
    ngpu_list.sort()

    results_table = np.zeros((len(ngpu_list), len(batch_size_list)))
    results_table[:] = np.nan
    df = pandas.DataFrame(results_table
                          , index=ngpu_list, columns=batch_size_list)

    for ilist in results:
        f, k, t, ts = ilist 
        b, g = k.split('-')
        df[int(b)][int(g)] = t
    df.replace(0, np.nan)

    fig = plt.figure(dpi=120)
    ax = fig.add_subplot(111)
    ax.set_title("Simple Results[{}]".format(_framework))
    for i in ngpu_list:
        ax.plot(df.iloc[i-1], 'o-')
    ax.legend([f"NGPU [{i}] "for i in ngpu_list])
    ax.set_ylabel("Total time [sec]")
    ax.set_xlabel("Batch Size")
    ax.grid(True)
    plt.show()

def compare_results(frameworks:list, _ngpu=1):
    p = Path("../results")

    target_gpu = "GeForce GTX 1080 Ti"
    results = []
    ngpu_set = set()
    batch_size_set = set()
    ignore_list = ["_sources", ".gitkeep"]
    for i in p.iterdir():
        if i.name in ignore_list:
            continue
        gpumodel = load_runinfo(i)
        if target_gpu != gpumodel:
            continue
        try:
            framework, batch_size, ngpu = load_config(i)
            if not framework in frameworks:
                continue
            if ngpu != _ngpu:
                continue
        except:
            continue
        try:
            ts, total = load_results(i)
        except:
            continue
        batch_size_set.add(batch_size)
        ngpu_set.add(ngpu)
        keyname = "{:06d}-{}".format(batch_size, ngpu)
        results.append([framework, keyname, total, ts])


    batch_size_list = list(batch_size_set) 
    batch_size_list.sort()
    ngpu_list = list(ngpu_set) 
    ngpu_list.sort()
    
    dfs = {}
    for iframework in frameworks:
        results_table = np.zeros((len(ngpu_list), len(batch_size_list)))
        results_table[:] = np.nan
        dfs[iframework] = pandas.DataFrame(results_table
                              , index=ngpu_list, columns=batch_size_list)
    
    for ilist in results:
        f, k, t, ts = ilist 
        b, g = k.split('-')
        dfs[f][int(b)][int(g)] = t


    fig = plt.figure(dpi=120)
    ax = fig.add_subplot(111)
    ax.set_title("Comparison Results[NGPU={}]".format(_ngpu))
    for iframework in frameworks:
        for i in ngpu_list:
            ax.plot(dfs[iframework].iloc[i-1], 'o-')
    ax.legend(frameworks)
    ax.set_ylabel("Total time [sec]")
    ax.set_xlabel("Batch Size")
    ax.grid(True)
    plt.show()

In [ ]:
### Simple results
get_simple_results('torch')
get_simple_results('chainer')


In [ ]:
# Comparison of frameworks
compare_results(['torch', 'chainer'], 1)